In [2]:
from py2neo import Graph, Node, Relationship

In [3]:
g=Graph("bolt://localhost:7687",password="1234")

#  Q1-RETRIEVE THE BARGAIN TRANSFERS 

In [3]:
q1 = '''MATCH (clubTo:Club)<-[r1:teamTo]-(transfer:Transfer)-[r:playerInvolved]->(player:Player)
WHERE toInteger(transfer.fee) < toInteger(r.MarketValue) and toInteger(transfer.fee) <> 0
RETURN player.name, toInteger(transfer.fee),toInteger(r.MarketValue), clubTo.name, toInteger(transfer.fee)* 1.0/toInteger(r.MarketValue) as div
ORDER BY div ASC
'''

In [4]:
g.run(q1).to_table()

player.name,toInteger(transfer.fee),toInteger(r.MarketValue),clubTo.name,div
Bernd Leno,502452,5000000,Bay. Leverkusen,0.1004904
Iago Falque,1004904,8000000,AS Roma,0.125613
Tin Jedvaj,1004904,6500000,Bay. Leverkusen,0.1546006153846154
Pizzi,1507349,7500000,Atl�tico Madrid,0.20097986666666667
Mapou Yanga-Mbiwa,1261707,6000000,AS Roma,0.2102845
Jos� Antonio Reyes,3517148,15000000,Sevilla FC,0.23447653333333332
Emmanuel Adebayor,6431357,25000000,Spurs,0.25725428
Maicon,3773956,14500000,Man City,0.2602728275862069
Fabio Borini,1261707,4800000,AS Roma,0.262855625
Nigel de Jong,3517148,13000000,AC Milan,0.27054984615384614


# Q2-Top3 buyer clubs per season. Note we used the extra edge created for the algorithms.

In [25]:
q2 = '''
MATCH (c2:Club)-[tr:transfer]->(c1:Club)<-[tto:teamTo]-(t:Transfer)-[ts:transferSeason]->(s:Season)
WITH s.year AS season, c1.name AS clubName, COUNT(DISTINCT ts) AS numberOfBuys
ORDER BY season, numberOfBuys DESC
WITH season, collect(clubName) AS clubList, collect(numberOfBuys) AS buys
UNWIND RANGE(0, 2) AS i
WITH season, [clubList[i], buys[i]] AS output
WITH season, COLLECT(output) AS output_list
RETURN season, output_list
ORDER BY season
'''


In [26]:
g.run(q2).to_table()

season,output_list
2009-2010,"[['Shakhtar D.', 2], ['Brescia', 1], ['Birmingham', 1]]"
2010-2011,"[['Genoa', 10], ['Man City', 7], ['Real Madrid', 6]]"
2011-2012,"[['Juventus', 8], ['AS Roma', 8], ['Paris SG', 7]]"
2012-2013,"[['Chelsea', 7], ['Juventus', 7], ['Man Utd', 6]]"
2013-2014,"[['SSC Napoli', 10], ['Monaco', 8], ['Spurs', 7]]"
2014-2015,"[['Liverpool', 8], ['Southampton', 8], ['Atl\ufffdtico Madrid', 7]]"
2015-2016,"[['Valencia CF', 8], ['Watford', 8], ['Monaco', 7]]"
2016-2017,"[['Inter', 9], ['Bor. Dortmund', 8], ['AS Roma', 7]]"
2017-2018,"[['AC Milan', 7], ['Juventus', 7], ['Everton', 5]]"


# Q3-Top transferred players:

In [16]:
q3 = '''MATCH (t:Transfer)-[ts:playerInvolved]->(p:Player)
RETURN p.name, COUNT(t) AS numberOfTransfers
ORDER BY numberOfTransfers DESC
'''

In [17]:
g.run(q3).to_table()

p.name,numberOfTransfers
Ciro Immobile,6
Fernando,5
Mattia Destro,5
Roberto,5
Andr� Sch�rrle,4
Christopher Samba,4
Danilo,4
Dimitri Payet,4
Dorlan Pab�n,4
Fabio Borini,4


# Q4-Most expensive positions:

In [18]:
q4 = '''
MATCH (transfer:Transfer)-[r:playerInvolved]->(player:Player) -[r1:playsAs]->(role:Role) 
RETURN role.position, avg(toInteger(transfer.fee)) as tf
ORDER BY tf DESC
'''

In [19]:
g.run(q4).to_table()

role.position,tf
Left Wing,15066724.106250001
Central Midfield,13239977.783251233
Right Wing,13039366.523178805
Attacking Midfield,12441405.285714285
Left Midfield,12314007.071428573
Centre-Forward,12257117.116704805
Centre-Back,11165594.385826776
Defensive Midfield,10900191.848101266
Left-Back,10570765.292134833
Right-Back,10536497.677419357


# RECCOMENDER SYSTEM: RETRIEVE ALL THE BARGAIN TRANSFERS FROM LESS THAN 3 AGES AND WITH PLAYERS YOUNGER THAN AN INPUT AGE. WE SUGGEST TO INPUT 21 OR 22.

In [21]:
x = (input())
queryRec = '''
MATCH (season:Season)<-[r1:transferSeason]-(transfer:Transfer)-[r:playerInvolved]->(player:Player)
WHERE toInteger(transfer.fee) < toInteger(r.MarketValue) and toInteger(transfer.fee) <> 0 and toInteger(r.Age)<'''+ x +''' and 2018 - toInteger(right(season.year,4)) <4 
RETURN player.name,r.Age,season.year, toInteger(transfer.fee),toInteger(r.MarketValue),  toInteger(transfer.fee)* 1.0/toInteger(r.MarketValue) as div
ORDER BY div ASC
'''

22


In [22]:
g.run(queryRec).to_table()

player.name,r.Age,season.year,toInteger(transfer.fee),toInteger(r.MarketValue),div
Tin Jedvaj,18,2014-2015,1004904,6500000,0.1546006153846154
Serge Gnabry,21,2017-2018,8039196,15000000,0.5359464
Domenico Berardi,20,2015-2016,10048995,17500000,0.5742282857142857
Massimo Bruno,20,2014-2015,5024497,8000000,0.628062125
D�ria,19,2014-2015,5024497,8000000,0.628062125
Luciano Vietto,20,2014-2015,5526947,8500000,0.6502290588235294
Daniele Rugani,20,2014-2015,5024497,7500000,0.6699329333333334
Josip Drmic,21,2014-2015,6833317,10000000,0.6833317
Michy Batshuayi,20,2014-2015,6029397,8000000,0.753674625
Fabinho,21,2015-2016,6029397,8000000,0.753674625


# ____________________________________


# Graph Algorithms

# PageRank

In [4]:
g.run(
'''
CALL algo.pageRank.stream("Club", "transfer", 
{iterations:20, dampingFactor:0.85})
YIELD nodeId, score
RETURN algo.asNode(nodeId).name AS club, score
ORDER BY score DESC
''').to_table()

club,score
Chelsea,5.703770010545848
Man Utd,4.6240362047217785
Juventus,4.47402398623526
Man City,4.4306454236619155
Liverpool,4.006158483214676
Everton,3.82435501255095
Newcastle,3.7802553507965064
Paris SG,3.7058295630384235
West Brom,3.6383956542238596
Valencia CF,3.612802034942433


# Betweenness Centrality

In [5]:
g.run(
'''
CALL algo.betweenness.stream('Club','transfer',{direction:'out'})
YIELD nodeId, centrality
RETURN algo.asNode(nodeId).name AS user,centrality
ORDER BY centrality DESC
''').to_table()

user,centrality
Chelsea,4618.868898918808
Juventus,4116.7037032357875
Atl�tico Madrid,3722.8112755090115
Inter,3422.340842734191
Spurs,3411.9503726129556
Monaco,2961.7188510874057
AS Roma,2847.023085752784
Man Utd,2629.1271993182236
Man City,2615.6721061321564
FC Porto,2577.2902233448235


# Closeness Centrality

In [6]:
g.run(
'''
CALL algo.closeness.stream('Club','transfer',{direction:'out'})
YIELD nodeId, centrality
RETURN algo.asNode(nodeId).name AS user,centrality
ORDER BY centrality DESC
''').to_table()

user,centrality
R. Madrid Youth,1.0
Depor Jugend,1.0
Benfica U19,1.0
Man City U18,1.0
SZ FC,1.0
Deportivo Cali,0.6666666666666666
YB Funde,0.6666666666666666
Chelsea,0.46956521739130436
Juventus,0.4682080924855491
Inter,0.46088193456614507


# Triangle Counting

In [14]:
g.run(
'''
CALL algo.triangle.stream('Club','transfer')
YIELD nodeA,nodeB,nodeC
RETURN algo.asNode(nodeA).name,algo.asNode(nodeB).name,algo.asNode(nodeC).name
''').to_table()

algo.asNode(nodeA).name,algo.asNode(nodeB).name,algo.asNode(nodeC).name
Ipswich,Sunderland,Stoke City
Ipswich,West Ham,Stoke City
Spurs,Man City,Sevilla FC
Spurs,Man City,FC Schalke 04
Spurs,Man City,Aston Villa
Spurs,Man City,AS Roma
Spurs,Man City,Valencia CF
Spurs,Man City,Atl�tico Madrid
Spurs,Man City,Villarreal CF
Spurs,Man City,Real Madrid


In [16]:
g.run(
'''
CALL algo.triangleCount.stream('Club', 'transfer', {concurrency:4})
YIELD nodeId, triangles, coefficient
RETURN algo.asNode(nodeId).name AS name, triangles, coefficient
ORDER BY coefficient DESC
''').to_table()

name,triangles,coefficient
Vit. Guimar�es,1,1.0
Dijon,1,1.0
BJ Renhe,1,1.0
Chiapas FC,1,1.0
Los Angeles,1,1.0
Toronto FC,1,1.0
Cagliari Calcio,4,0.6666666666666666
Celta de Vigo,2,0.6666666666666666
KAA Gent,2,0.6666666666666666
FC Empoli,6,0.6


# Connected Components

In [8]:
g.run(
'''
CALL algo.unionFind.stream("Club", "transfer")
YIELD nodeId, setId
RETURN algo.asNode(nodeId) AS Name, setId AS ComponentId
ORDER BY ComponentId, Name
''').to_table()

Name,ComponentId
(_6351:Club {name: 'Rayo Vallecano'}),2
(_6411:Club {name: 'Ipswich'}),2
(_6412:Club {name: 'Spurs'}),2
(_6413:Club {name: 'Man City'}),2
(_6414:Club {name: 'Al-Jazira'}),2
(_6415:Club {name: 'Getafe CF'}),2
(_6416:Club {name: 'Genoa'}),2
(_6417:Club {name: 'Sevilla FC'}),2
(_6418:Club {name: 'Stade Rennais'}),2
(_6419:Club {name: 'FC Augsburg'}),2


In [9]:
g.run(
'''
CALL algo.unionFind("Club", "transfer", {
  write: true,
  writeProperty: "componentId"
})
YIELD nodes AS Nodes, setCount AS NbrOfComponents, writeProperty AS PropertyName
''').to_table()

Nodes,NbrOfComponents,PropertyName
332,4,componentId


# Strongly Connected Components

In [10]:
g.run(
'''
CALL algo.scc.stream("Club", "transfer")
YIELD nodeId, partition
RETURN algo.asNode(nodeId) AS Name, partition AS ComponentId
ORDER BY ComponentId, Name
''').to_table()

Name,ComponentId
"(_6351:Club {componentId: 2, name: 'Rayo Vallecano'})",0
"(_6411:Club {componentId: 2, name: 'Ipswich'})",1
"(_6419:Club {componentId: 2, name: 'FC Augsburg'})",9
"(_6424:Club {componentId: 2, name: 'AZ Alkmaar'})",14
"(_6429:Club {componentId: 2, name: 'FC Barcelona B'})",19
"(_6433:Club {componentId: 2, name: 'Charlton'})",23
"(_6434:Club {componentId: 2, name: 'Braga'})",24
"(_6412:Club {componentId: 2, name: 'Spurs'})",29
"(_6413:Club {componentId: 2, name: 'Man City'})",29
"(_6414:Club {componentId: 2, name: 'Al-Jazira'})",29


In [11]:
g.run(
'''
CALL algo.scc("Club", "transfer", {
  write: true
})
YIELD nodes AS Nodes, setCount AS NbrOfComponents
''').to_table()

Nodes,NbrOfComponents
332,188


# Louvain Modularity

In [12]:
g.run(
'''
CALL algo.louvain.stream('Club', 'transfer', {
 graph: 'huge',
 direction: 'BOTH'
}) YIELD nodeId, community, communities
RETURN algo.asNode(nodeId).name as name, community, communities
ORDER BY name ASC
''').to_table()

name,community,communities
1. FC K�ln,2,null
1.FC K'lautern,2,null
1.FC Nuremberg,2,null
1.FSV Mainz 05,2,null
AC Le Havre,3,null
AC Milan,4,null
AFC Ajax,1,null
AIK Solna,2,null
AJ Auxerre,3,null
AS Nancy,6,null


In [13]:
g.run(
'''
CALL algo.louvain('Club', 'transfer', {
 direction: 'BOTH',
 writeProperty: 'community'
}) YIELD  communityCount, modularity, modularities
''').to_table()

communityCount,modularity,modularities
12,0.3361143838076322,"[0.3491193438107762, 0.339428776077543, 0.3361143838076322]"
